# Stage 1. Discovering Bangkok Neighbourhoods

In [1]:
import requests
import numpy as np # library to handle data in a vectorized manner
import os

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Installing BeautifulSoup4 (if not installed yet - uncommend the line below)
#!pip install BeautifulSoup4
from bs4 import BeautifulSoup
from datetime import datetime as dt
import json # library to handle JSON files
import sqlite3

print('\n','>>> Libraries imported.')


 >>> Libraries imported.


## 1. Getting the list of all Bangkok neighbourhoods (khwaengs)

From Wikipedia page <a href="https://en.wikipedia.org/wiki/Khwaeng">"Khwaeng"</a>

Downloading the page from the Web:

In [2]:
link = 'https://en.wikipedia.org/wiki/Khwaeng'
page = requests.get(link)
print(page.status_code)

200


Cooking BeautifulSoup from the retireved page :

In [3]:
soup = BeautifulSoup(page.text, 'html.parser')

The data of interest are in a table, so get tables from the soup:

In [4]:
tables = soup('table')
len(tables)

2

There are two tables on the page. The khwaengs are listed in the first one:

In [5]:
khwaengs_table = tables[0]

row_tags = khwaengs_table('tr')
for tag in row_tags[:9]:
    print(tag)
    print(tag.text, end = "\n"*2)

<tr>
<th colspan="3">District (<i>khet</i>)
</th>
<th colspan="3">Subdistrict (<i>khwaeng</i>)
</th>
<th rowspan="2">Notes
</th></tr>

District (khet)

Subdistrict (khwaeng)

Notes


<tr>
<th>Code
</th>
<th>Name
</th>
<th>Name (Thai)
</th>
<th>Code
</th>
<th>Name
</th>
<th>Name (Thai)
</th></tr>

Code

Name

Name (Thai)

Code

Name

Name (Thai)


<tr>
<td rowspan="12" valign="top">01
</td>
<td rowspan="12" valign="top"><a href="/wiki/Phra_Nakhon_district" title="Phra Nakhon district">Phra Nakhon</a>
</td>
<td rowspan="12" valign="top">พระนคร
</td>
<td>01
</td>
<td><a href="/wiki/Phra_Borom_Maha_Ratchawang_subdistrict" title="Phra Borom Maha Ratchawang subdistrict">Phra Borom Maha Ratchawang</a>
</td>
<td>พระบรมมหาราชวัง
</td>
<td>
</td></tr>

01

Phra Nakhon

พระนคร

01

Phra Borom Maha Ratchawang

พระบรมมหาราชวัง




<tr>
<td>02
</td>
<td><a href="/wiki/Wang_Burapha_Phirom_subdistrict" title="Wang Burapha Phirom subdistrict">Wang Burapha Phirom</a>
</td>
<td>วังบูรพาภิรมย์
</td>
<td>


There are total of 6 columns, and the first two rows are header.

Parse the Khwaengs table into a list of rows:

In [6]:
rows = list()
for tag in row_tags:
    row = tag.text.strip('\n').split('\n\n')
    rows.append(row)

print('Top 9 rows:')
for row in rows[0:9] :
    print(row)
print('Bottom 9 rows:')
for row in rows[-9:] :
    print(row)
print()
print('Total rows including the headers:', len(rows))

Top 9 rows:
['District (khet)', 'Subdistrict (khwaeng)', 'Notes']
['Code', 'Name', 'Name (Thai)', 'Code', 'Name', 'Name (Thai)']
['01', 'Phra Nakhon', 'พระนคร', '01', 'Phra Borom Maha Ratchawang', 'พระบรมมหาราชวัง']
['02', 'Wang Burapha Phirom', 'วังบูรพาภิรมย์']
['03', 'Wat Ratchabophit', 'วัดราชบพิธ']
['04', 'Samran Rat', 'สำราญราษฎร์']
['05', 'San Chaopho Suea', 'ศาลเจ้าพ่อเสือ']
['06', 'Sao Chingcha', 'เสาชิงช้า', 'Seat of BMA office']
['07', 'Bowon Niwet', 'บวรนิเวศ']
Bottom 9 rows:
['03', 'Bang Na Tai', 'บางนาใต้']
['48', 'Thawi Watthana', 'ทวีวัฒนา', '01', 'Thawi Watthana', 'ทวีวัฒนา', 'District seat']
['02', 'Sala Thammasop', 'ศาลาธรรมสพน์']
['49', 'Thung Khru', 'ทุ่งครุ', '01', 'Bang Mot', 'บางมด']
['02', 'Thung Khru', 'ทุ่งครุ', 'District seat']
['50', 'Bang Bon', 'บางบอน', '02', 'Bang Bon Nuea', 'บางบอนเหนือ']
['03', 'Bang Bon Tai', 'บางบอนใต้', 'District seat']
['04', 'Khlong Bang Phran', 'คลองบางพราน']
['05', 'Khlong Bang Bon', 'คลองบางบอน']

Total rows including the heade

Making all rows uniform by adding district data to rows which only contain neighbourhjood data as a result of the complex table layout, while excluding the header:

## 2. Persisting the data into SQLite database.

- Setting up the database:

In [7]:
connection = sqlite3.connect('db/bkk_khwaengs.sqlite')
cursor = connection.cursor()

In [10]:
# cursor.execute('DROP TABLE IF EXISTS Khet')
cursor.execute('''CREATE TABLE IF NOT EXISTS Khet (
    idKhet INTEGER NOT NULL UNIQUE PRIMARY KEY AUTOINCREMENT,
    KhetCode CHAR(2) NOT NULL UNIQUE, 
    KhetEng VARCHAR(64) NOT NULL UNIQUE,
    KhetThai VARCHAR(64) NOT NULL UNIQUE
    )'''
)

# cursor.execute('DROP TABLE IF EXISTS Khwaeng')
cursor.execute('''CREATE TABLE IF NOT EXISTS Khwaeng (
    idKhwaeng INTEGER NOT NULL UNIQUE PRIMARY KEY AUTOINCREMENT, 
    KhwaengCode CHAR(2) NOT NULL,
    KhwaengEng VARCHAR(64) NOT NULL,
    KhwaengThai VARCHAR(64) NOT NULL,
    Latitude FLOAT,
    Longitude FLOAT,
    idKhet INTEGER, 
    FOREIGN KEY (idKhet) REFERENCES Khet (idKhet) ON DELETE CASCADE ON UPDATE CASCADE
    )'''
)

connection.commit()

- Populating the Database with data row by row

In [11]:
for row in rows[2:]:  # skip the header
    if len(row) >= len(rows[2]) :  # the first complete row as a reference
        cursor.execute('INSERT INTO Khet (KhetCode, KhetEng, KhetThai) VALUES(?, ?, ?)',
                          tuple(row[:3]))  # leading cells of a complete row go into Khet table
        idKhet = cursor.lastrowid  # to be used as foreign key in Khwaeng table
        cursor.execute('INSERT INTO Khwaeng (KhwaengCode, KhwaengEng, KhwaengThai, idKhet) VALUES(?, ?, ?, ?)',
                  (row[3], row[4], row[5], idKhet))
    else:
        cursor.execute('INSERT INTO Khwaeng (KhwaengCode, KhwaengEng, KhwaengThai, idKhet) VALUES(?, ?, ?, ?)',
                  (row[0], row[1], row[2], idKhet))

connection.commit()

- Checking the result

In [16]:
sql = '''SELECT Khet.KhetCode, Khet.KhetEng, Khet.KhetThai, Khwaeng.KhwaengCode, Khwaeng.KhwaengEng, Khwaeng.KhwaengThai
            FROM Khet JOIN Khwaeng on Khwaeng.idKhet = Khet.idKhet'''

In [20]:
select = cursor.execute(sql)

In [21]:
data = select.fetchall()
data[:11]

[('01',
  'Phra Nakhon',
  'พระนคร',
  '01',
  'Phra Borom Maha Ratchawang',
  'พระบรมมหาราชวัง'),
 ('01',
  'Phra Nakhon',
  'พระนคร',
  '02',
  'Wang Burapha Phirom',
  'วังบูรพาภิรมย์'),
 ('01', 'Phra Nakhon', 'พระนคร', '03', 'Wat Ratchabophit', 'วัดราชบพิธ'),
 ('01', 'Phra Nakhon', 'พระนคร', '04', 'Samran Rat', 'สำราญราษฎร์'),
 ('01', 'Phra Nakhon', 'พระนคร', '05', 'San Chaopho Suea', 'ศาลเจ้าพ่อเสือ'),
 ('01', 'Phra Nakhon', 'พระนคร', '06', 'Sao Chingcha', 'เสาชิงช้า'),
 ('01', 'Phra Nakhon', 'พระนคร', '07', 'Bowon Niwet', 'บวรนิเวศ'),
 ('01', 'Phra Nakhon', 'พระนคร', '08', 'Talat Yot', 'ตลาดยอด'),
 ('01', 'Phra Nakhon', 'พระนคร', '09', 'Chana Songkhram', 'ชนะสงคราม'),
 ('01', 'Phra Nakhon', 'พระนคร', '10', 'Ban Phan Thom', 'บ้านพานถม'),
 ('01', 'Phra Nakhon', 'พระนคร', '11', 'Bang Khun Phrom', 'บางขุนพรหม')]

In [14]:
data[-11:]

[('46',
  'Khlong Sam Wa',
  'คลองสามวา',
  '05',
  'Sai Kong Din Tai',
  'ทรายกองดินใต้'),
 ('47', 'Bang Na', 'บางนา', '02', 'Bang Na Nuea', 'บางนาเหนือ'),
 ('47', 'Bang Na', 'บางนา', '03', 'Bang Na Tai', 'บางนาใต้'),
 ('48', 'Thawi Watthana', 'ทวีวัฒนา', '01', 'Thawi Watthana', 'ทวีวัฒนา'),
 ('48', 'Thawi Watthana', 'ทวีวัฒนา', '02', 'Sala Thammasop', 'ศาลาธรรมสพน์'),
 ('49', 'Thung Khru', 'ทุ่งครุ', '01', 'Bang Mot', 'บางมด'),
 ('49', 'Thung Khru', 'ทุ่งครุ', '02', 'Thung Khru', 'ทุ่งครุ'),
 ('50', 'Bang Bon', 'บางบอน', '02', 'Bang Bon Nuea', 'บางบอนเหนือ'),
 ('50', 'Bang Bon', 'บางบอน', '03', 'Bang Bon Tai', 'บางบอนใต้'),
 ('50', 'Bang Bon', 'บางบอน', '04', 'Khlong Bang Phran', 'คลองบางพราน'),
 ('50', 'Bang Bon', 'บางบอน', '05', 'Khlong Bang Bon', 'คลองบางบอน')]

In [15]:
len(data)

180

## 3. Pulling the data into Pandas DataFrame

In [23]:
khwaengs = pd.read_sql(sql, connection)
khwaengs.head(22)

,KhetCode,KhetEng,KhetThai,KhwaengCode,KhwaengEng,KhwaengThai
0,01,Phra Nakhon,พระนคร,01,Phra Borom Maha Ratchawang,พระบรมมหาราชวัง
1,01,Phra Nakhon,พระนคร,02,Wang Burapha Phirom,วังบูรพาภิรมย์
2,01,Phra Nakhon,พระนคร,03,Wat Ratchabophit,วัดราชบพิธ
3,01,Phra Nakhon,พระนคร,04,Samran Rat,สำราญราษฎร์
4,01,Phra Nakhon,พระนคร,05,San Chaopho Suea,ศาลเจ้าพ่อเสือ
5,01,Phra Nakhon,พระนคร,06,Sao Chingcha,เสาชิงช้า
6,01,Phra Nakhon,พระนคร,07,Bowon Niwet,บวรนิเวศ
7,01,Phra Nakhon,พระนคร,08,Talat Yot,ตลาดยอด
8,01,Phra Nakhon,พระนคร,09,Chana Songkhram,ชนะสงคราม
9,01,Phra Nakhon,พระนคร,10,Ban Phan Thom,บ้านพานถม


In [18]:
khwaengs.dtypes

KhetCode       object
KhetEng        object
KhetThai       object
KhwaengCode    object
KhwaengEng     object
KhwaengThai    object
dtype: object

In [19]:
khwaengs.describe()

,KhetCode,KhetEng,KhetThai,KhwaengCode,KhwaengEng,KhwaengThai
count,180,180,180,180,180,180
unique,50,50,50,12,178,178
top,01,Phra Nakhon,พระนคร,02,Bang Chak,บางจาก
freq,12,12,12,43,2,2


Applying the .describe() method to the dataframe instantly reveals an anomaly: the total count of neghbourhoods is 180 but there are only 178 unique entries. One of the most frequently occuring neighbourhood names is Bang Chak, and the top frequency is 2. So they are likely 2 duplicates.

Checking the data for duplicate khwaeng names:

In [25]:
dup = khwaengs[khwaengs.duplicated(subset='KhwaengEng', keep=False)]
dup

,KhetCode,KhetEng,KhetThai,KhwaengCode,KhwaengEng,KhwaengThai
43,09,Phra Khanong,พระโขนง,05,Bang Chak,บางจาก
91,22,Phasi Charoen,ภาษีเจริญ,06,Bang Chak,บางจาก
133,35,Chom Thong,จอมทอง,03,Bang Mot,บางมด
174,49,Thung Khru,ทุ่งครุ,01,Bang Mot,บางมด


And indeed there are 2 pairs of same-name khwaengs belonging to different districts

Which is probably not an issue as they are relationally bound to respective khets (i.e. districts).

In [ ]:
connection.close()